# Parametrizing Panoptic Segmentation

In [1]:
%load_ext autoreload
%autoreload 2

import os
import sys

# Add the project's files to the python path
# file_path = os.path.dirname(os.path.dirname(os.path.abspath(__file__)))  # for .py script
file_path = os.path.dirname(os.path.abspath(''))  # for .ipynb notebook
sys.path.append(file_path)

import hydra
from src.utils import init_config, compute_panoptic_metrics, \
    grid_search_panoptic_partition, oracle_superpoint_clustering, \
    grid_search_panoptic_partition_FORinstance, panoptic_partition_FORinstance, \
    bayesian_optimize_panoptic_partition_FORinstance
import torch
from src.transforms import *
from src.utils.widgets import *
from src.data import *

# Very ugly fix to ignore lightning's warning messages about the
# trainer and modules not being connected
import warnings
warnings.filterwarnings("ignore")

In [2]:
device = 'cuda' 
# change for each experiment

experiment = 'forinstance_100edgeaffloss_5e-2LR_2e-1voxel_20kadj_800epochs_panoptic1'
ckpt = '/home/valerio/Downloads/100edgeaffloss_5e-2LR_2e-1voxel_20kadj_800epochs_panoptic1.ckpt'

stage = 'test'

In [3]:
# Parse the configs using hydra
cfg = init_config(overrides=[
    f"experiment=panoptic/{experiment}",
    f"ckpt_path={ckpt}"
])

datamodule = hydra.utils.instantiate(cfg.datamodule)
datamodule.prepare_data()
datamodule.setup()

if stage == 'val':
    dataset = datamodule.val_dataset
elif stage == 'train':
    dataset = datamodule.train_dataset
elif stage == 'test':
    dataset = datamodule.test_dataset
else:
    raise ValueError("Invalid stage")

dataset.print_classes()

model = hydra.utils.instantiate(cfg.model)

if ckpt is not None:
    model = model._load_from_checkpoint(cfg.ckpt_path)

model = model.eval().to(device)

0   Ground and low vegetation stuff
1   Tree                 thing
2   Unknown              void


In [24]:
tile = 10
grid_search = False

if grid_search:
    k_max = [10, 20]
    radius = [1, 3, 5]
    regularization = [10, 20]
    x_weight = [1e-2, 1e-1, 1]
    cutoff = [1000, 100]
else:
    k_max = [10]
    radius = [3]
    regularization = [20]
    x_weight = [1e-1]
    cutoff = [1000]

best_output, best_partitions, best_results, best_combination = grid_search_panoptic_partition_FORinstance(
    model,
    dataset,
    i_cloud=tile,
    graph_kwargs=dict(
        k_max=k_max,
        radius=radius),
    partition_kwargs=dict(
        regularization=regularization,
        x_weight=x_weight,
        cutoff=cutoff),
    mode='pas',
    verbose=False)

Param Combinations: 100%|██████████| 1/1 [00:00<00:00,  3.28it/s]


Best panoptic setup: 
Tree F1=36.59
   k_max  radius  regulari.  x_weight  cutoff
0     10       3         20       0.1    1000



In [25]:
if False:
    nag = dataset[tile]

    # add the results to the nag for visualization
    vox_y, vox_index, vox_obj_pred = best_output.voxel_panoptic_pred(super_index=nag[0].super_index)
    nag[0].obj_pred = vox_obj_pred

    center = nag[0].pos.mean(dim=0)[:2]
    radius = 10
    mask = torch.where(torch.linalg.norm(nag[0].pos[:, :2] - center, dim=1) < radius)[0]
    nag_cylinder = nag.select(0, mask)
    nag_cylinder.show(path='/home/valerio/Documents/Semester_Thesis/visualizations/', title='TUWIEN', stuff_classes=[0], num_classes=2)

In [4]:
panoptic, instance, semantic = compute_panoptic_metrics(
    model,
    datamodule,
    stage='test',
    graph_kwargs=dict(
        k_max=10,
        radius=3),
    partition_kwargs=dict(
        regularization=20,
        x_weight=1e-1,
        cutoff=1000))

100%|██████████| 3/3 [00:03<00:00,  1.06s/it]

pq                    : 0.6818211674690247
sq                    : 0.8860996961593628
rq                    : 0.7544098496437073
pq_modified           : 0.7045230269432068
pq_thing              : 0.45556876063346863
sq_thing              : 0.8187221884727478
rq_thing              : 0.5564387440681458
pq_stuff              : 0.9080736041069031
sq_stuff              : 0.9534772634506226
rq_stuff              : 0.9523809552192688
pq_per_class          : tensor([0.9081, 0.4556])
sq_per_class          : tensor([0.9535, 0.8187])
rq_per_class          : tensor([0.9524, 0.5564])
precision_per_class   : tensor([0.9091, 0.5738])
recall_per_class      : tensor([1.0000, 0.5401])
tp_per_class          : tensor([ 10, 175])
fp_per_class          : tensor([  1, 130])
fn_per_class          : tensor([  0, 149])
pq_modified_per_class : tensor([0.9535, 0.4556])
mean_precision        : 0.7414306998252869
mean_recall           : 0.770061731338501
mIoU                  : 97.24026489257812
